In [18]:
import glob
import math
import os.path
import re
import sys
#import r"C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\cityscapes\build_data.py"
import tensorflow as tf

In [36]:
cityscapes_root = r"C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\cityscapes"

output_dir = r"C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\cityscapes\tfrecord"

_NUM_SHARDS = 10

_FOLDERS_MAP = {
    'image': 'leftImg8bit',
    'label': 'gtFine',
}

_POSTFIX_MAP = {
    'image': '_leftImg8bit',
    'label': '_gtFine_labelTrainIds',
}

_DATA_FORMAT_MAP = {
    'image': 'png',
    'label': 'png',
}

# Image file pattern.
_IMAGE_FILENAME_RE = re.compile('(.+)' + _POSTFIX_MAP['image'])

In [ ]:
def _get_files(data, dataset_split):
    pattern = '*%s.%s' % (_POSTFIX_MAP[data], _DATA_FORMAT_MAP[data])
    search_files = os.path.join(
      cityscapes_root, _FOLDERS_MAP[data], dataset_split, '*', pattern)
    filenames = glob.glob(search_files)
    return sorted(filenames)

_get_files("label", "train")

In [38]:
def _convert_dataset(dataset_split):
    image_files = _get_files('image', dataset_split)
    label_files = _get_files('label', dataset_split)

    num_images = len(image_files)
    num_per_shard = int(math.ceil(num_images / float(_NUM_SHARDS)))

    image_reader = ImageReader('png', channels=3)#build_data.ImageReader('png', channels=3)
    label_reader = ImageReader('png', channels=1)#build_data.ImageReader('png', channels=1)
    
    for shard_id in range(_NUM_SHARDS):
        shard_filename = '%s-%05d-of-%05d.tfrecord' % (
            dataset_split, shard_id, _NUM_SHARDS)
        output_filename = os.path.join(output_dir, shard_filename)
        with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
            start_idx = shard_id * num_per_shard
            end_idx = min((shard_id + 1) * num_per_shard, num_images)
            for i in range(start_idx, end_idx):
                sys.stdout.write('\r>> Converting image %d/%d shard %d' % (
                i + 1, num_images, shard_id))
                sys.stdout.flush()
                # Read the image.
                image_data = tf.gfile.FastGFile(image_files[i], 'rb').read()
                height, width = image_reader.read_image_dims(image_data)
                # Read the semantic segmentation annotation.
                seg_data = tf.gfile.FastGFile(label_files[i], 'rb').read()
                seg_height, seg_width = label_reader.read_image_dims(seg_data)
                if height != seg_height or width != seg_width:
                    raise RuntimeError('Shape mismatched between image and label.')
                # Convert to tf example.
                re_match = _IMAGE_FILENAME_RE.search(image_files[i])
                if re_match is None:
                    raise RuntimeError('Invalid image filename: ' + image_files[i])
                filename = os.path.basename(re_match.group(1))
                example = image_seg_to_tfexample(
                    image_data, filename, height, width, seg_data)
                tfrecord_writer.write(example.SerializeToString())
        sys.stdout.write('\n')
        sys.stdout.flush()

In [55]:
_convert_dataset('train')

>> Converting image 298/2975 shard 0
>> Converting image 596/2975 shard 1
>> Converting image 894/2975 shard 2
>> Converting image 1192/2975 shard 3
>> Converting image 1490/2975 shard 4
>> Converting image 1788/2975 shard 5
>> Converting image 2086/2975 shard 6
>> Converting image 2384/2975 shard 7
>> Converting image 2682/2975 shard 8
>> Converting image 2975/2975 shard 9


In [56]:
_convert_dataset('val')

>> Converting image 50/500 shard 0
>> Converting image 100/500 shard 1
>> Converting image 150/500 shard 2
>> Converting image 200/500 shard 3
>> Converting image 250/500 shard 4
>> Converting image 300/500 shard 5
>> Converting image 350/500 shard 6
>> Converting image 400/500 shard 7
>> Converting image 450/500 shard 8
>> Converting image 500/500 shard 9


In [54]:
# Copyright 2018 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Contains common utility functions and classes for building dataset.

This script contains utility functions and classes to converts dataset to
TFRecord file format with Example protos.

The Example proto contains the following fields:

  image/encoded: encoded image content.
  image/filename: image filename.
  image/format: image file format.
  image/height: image height.
  image/width: image width.
  image/channels: image channels.
  image/segmentation/class/encoded: encoded semantic segmentation content.
  image/segmentation/class/format: semantic segmentation file format.
"""
import collections
import six
import tensorflow as tf

FLAGS = tf.app.flags.FLAGS

#tf.app.flags.DEFINE_enum('image_format', 'png', ['jpg', 'jpeg', 'png'],'Image format.')

#Added by Casey
image_format = "png"

#tf.app.flags.DEFINE_enum('label_format', 'png', ['png'], 'Segmentation label format.')
label_format = "png"

# A map from image format to expected data format.
_IMAGE_FORMAT_MAP = {'jpg': 'jpeg', 'jpeg': 'jpeg', 'png': 'png'}


class ImageReader(object):
  """Helper class that provides TensorFlow image coding utilities."""

  def __init__(self, image_format='jpeg', channels=3):
    """Class constructor.

    Args:
      image_format: Image format. Only 'jpeg', 'jpg', or 'png' are supported.
      channels: Image channels.
    """
    with tf.Graph().as_default():
      self._decode_data = tf.placeholder(dtype=tf.string)
      self._image_format = image_format
      self._session = tf.Session()
      if self._image_format in ('jpeg', 'jpg'):
        self._decode = tf.image.decode_jpeg(self._decode_data,
                                            channels=channels)
      elif self._image_format == 'png':
        self._decode = tf.image.decode_png(self._decode_data, channels=channels)

  def read_image_dims(self, image_data):
    """Reads the image dimensions.

    Args:
      image_data: string of image data.

    Returns:
      image_height and image_width.
    """
    image = self.decode_image(image_data)
    return image.shape[:2]

  def decode_image(self, image_data):
    """Decodes the image data string.

    Args:
      image_data: string of image data.

    Returns:
      Decoded image data.

    Raises:
      ValueError: Value of image channels not supported.
    """
    image = self._session.run(self._decode, feed_dict={self._decode_data: image_data})
    if len(image.shape) != 3 or image.shape[2] not in (1, 3):
      raise ValueError('The image channels not supported.')

    return image


def _int64_list_feature(values):
  """Returns a TF-Feature of int64_list.

  Args:
    values: A scalar or list of values.

  Returns:
    A TF-Feature.
  """
  if not isinstance(values, collections.Iterable):
    values = [values]

  return tf.train.Feature(int64_list=tf.train.Int64List(value=values))


def _bytes_list_feature(values):
  """Returns a TF-Feature of bytes.

  Args:
    values: A string.

  Returns:
    A TF-Feature.
  """
  def norm2bytes(value):
    return value.encode() if isinstance(value, str) and six.PY3 else value

  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[norm2bytes(values)]))


def image_seg_to_tfexample(image_data, filename, height, width, seg_data):
  """Converts one image/segmentation pair to tf example.

  Args:
    image_data: string of image data.
    filename: image filename.
    height: image height.
    width: image width.
    seg_data: string of semantic segmentation data.

  Returns:
    tf example of one image/segmentation pair.
  """
  return tf.train.Example(features=tf.train.Features(feature={
      'image/encoded': _bytes_list_feature(image_data),
      'image/filename': _bytes_list_feature(filename),
      'image/format': _bytes_list_feature(_IMAGE_FORMAT_MAP[image_format]),
      'image/height': _int64_list_feature(height),
      'image/width': _int64_list_feature(width),
      'image/channels': _int64_list_feature(3),
      'image/segmentation/class/encoded': (_bytes_list_feature(seg_data)),
      'image/segmentation/class/format': _bytes_list_feature(label_format),
  }))
